In [1]:
import requests
import json
from folium import Map, TileLayer
from io import BytesIO
import rasterio
import urllib

%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [ ]:
# For categorical data:
gdal_translate -of COG -co TILING_SCHEME=GoogleMapsCompatible -co COMPRESS=LZW -co RESAMPLING=NEAREST -co OVERVIEW_RESAMPLING=MODE -co WARP_RESAMPLING=NEAREST -co OVERVIEWS=IGNORE_EXISTING ./masked_globcover.tif ./globcover_cog_test.tif

In [ ]:
# For continous data:
gdal_translate -of COG -co TILING_SCHEME=GoogleMapsCompatible -co COMPRESS=LZW -co RESAMPLING=BILINEAR -co OVERVIEW_RESAMPLING=BILINEAR -co WARP_RESAMPLING=NEAREST -co OVERVIEWS=IGNORE_EXISTING ./spam2017V2r1_SSA_H_WHEA_A.tif ./spam_wheat_cog.tif

In [18]:
titiler_endpoint = "https://tt.resat.dev-vizzuality.com"  
# titiler_endpoint = "https://cog-cors.tt.resat.dev-vizzuality.com/"
# titiler_endpoint = "http://127.0.0.1:3000"
# Two sample COGs - one being a PSScene analytic image and the other being a PSOrthoTile visual image
# from the Google Cloud storage bucket
cog_url = "https://resilience-atlas.s3.amazonaws.com/cog/globcover_cog_test.tif"
# cog_url = "https://oin-hotosm.s3.amazonaws.com/56f9b5a963ebf4bc00074e70/0/56f9c2d42b67227a79b4faec.tif"
# https://resilience-atlas.s3.amazonaws.com/cog/spam_wheat_cog.tif
geometry = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"coordinates":[[[33.17720646412923,9.16050898883553],[33.17720646412923,7.590139486949454],[34.958987658993124,7.590139486949454],[34.958987658993124,9.16050898883553],[33.17720646412923,9.16050898883553]]],"type":"Polygon"}}]}

In [28]:
def build_url(BASE_URL, endpoint, params):
    # , safe='{}/:"\'#', quote_via=urllib.parse.quote
    str_params = urllib.parse.urlencode(params)
    return f"{BASE_URL}/{endpoint}?{str_params}"

def submit_request(BASE_URL, endpoint, url):
    r = requests.get(
        f"{BASE_URL}/{endpoint}",
        params = {
            "url": url,
        }
    ).json()
    return r

def submit_analysis(BASE_URL, endpoint, url, geometry):
    r = requests.post(
        f"{BASE_URL}/{endpoint}",
        params = {"url": url},
        body = {"geometry": geometry}
    ).json()
    return r

In [20]:
analytic_cog_validation = submit_request(titiler_endpoint, "info", cog_url)
print(json.dumps(analytic_cog_validation, indent=1))

{
 "bounds": [
  -17.57812499999999,
  -11.178401873711785,
  141.32812500000003,
  38.82259097617711
 ],
 "minzoom": 2,
 "maxzoom": 9,
 "band_metadata": [
  [
   "b1",
   {}
  ]
 ],
 "band_descriptions": [
  [
   "b1",
   ""
  ]
 ],
 "dtype": "uint8",
 "nodata_type": "Nodata",
 "colorinterp": [
  "gray"
 ],
 "nodata_value": 0.0,
 "height": 19456,
 "count": 1,
 "width": 57856,
 "overviews": [
  2,
  4,
  8,
  16,
  32,
  64,
  128,
  256
 ],
 "driver": "GTiff"
}


In [21]:
tile_json_analytic = submit_request(titiler_endpoint, "tilejson.json", cog_url)
print(json.dumps(tile_json_analytic, indent=4))

{
    "tilejson": "2.2.0",
    "version": "1.0.0",
    "scheme": "xyz",
    "tiles": [
        "https://tt.resat.dev-vizzuality.com/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?url=https%3A%2F%2Fresilience-atlas.s3.amazonaws.com%2Fcog%2Fglobcover_cog_test.tif"
    ],
    "minzoom": 2,
    "maxzoom": 9,
    "bounds": [
        -17.57812499999999,
        -11.178401873711785,
        141.32812500000003,
        38.82259097617711
    ],
    "center": [
        61.87500000000002,
        13.822094551232663,
        2
    ]
}


In [30]:
# "colormap": json.dumps({"11":"#AAF0F0","14":"#FFFF64","20":"#DCF064","30":"#CDCD66","40":"#006400","50":"#00A000","60":"#AAC800","70": "#003C00","90":"#286400","100":"#788200","110":"#8CA000","120":"#BE9600","130":"#966400","140":"#FFB432","150":"#FFEBAF","160":"#00785A","170":"#009678","180":"#00DC82","190":"#C31400","200":"#FFF5D7","210":"#0046C8","220":"#FFFFFF","230":"#000000"}),
#    "colormap_type":"linear"
colormap = json.dumps({"11":"#AAF0F0","14":"#FFFF64","20":"#DCF064","30":"#CDCD66","40":"#006400","50":"#00A000","60":"#AAC800","70": "#003C00","90":"#286400","100":"#788200","110":"#8CA000","120":"#BE9600","130":"#966400","140":"#FFB432","150":"#FFEBAF","160":"#00785A","170":"#009678","180":"#00DC82","190":"#C31400","200":"#FFF5D7","210":"#0046C8","220":"#FFFFFF","230":"#000000"})
params = {
    "url": cog_url,
    "colormap": colormap,
    
}
tileset = build_url(titiler_endpoint, "tiles/{z}/{x}/{y}.png", params)
#tileset = tile_json_analytic.get("tiles", [])[0]
print(tileset)
(left, bottom, right, top) = tile_json_analytic["bounds"]
(lon,lat,zoom) = tile_json_analytic.get("center", [0, 0, 2])
center = [lat,lon]
m = Map(
    location=center,
    zoom_start=zoom,
    tiles="OpenStreetMap"
)
tile_layer = TileLayer(
    tiles= tileset,
    attr="Sample COG"
)

tile_layer.add_to(m)
m

{"11": "#AAF0F0", "14": "#FFFF64", "20": "#DCF064", "30": "#CDCD66", "40": "#006400", "50": "#00A000", "60": "#AAC800", "70": "#003C00", "90": "#286400", "100": "#788200", "110": "#8CA000", "120": "#BE9600", "130": "#966400", "140": "#FFB432", "150": "#FFEBAF", "160": "#00785A", "170": "#009678", "180": "#00DC82", "190": "#C31400", "200": "#FFF5D7", "210": "#0046C8", "220": "#FFFFFF", "230": "#000000"}
https://tt.resat.dev-vizzuality.com/tiles/{z}/{x}/{y}.png?url=https%3A%2F%2Fresilience-atlas.s3.amazonaws.com%2Fcog%2Fglobcover_cog_test.tif&colormap=%7B%2211%22%3A+%22%23AAF0F0%22%2C+%2214%22%3A+%22%23FFFF64%22%2C+%2220%22%3A+%22%23DCF064%22%2C+%2230%22%3A+%22%23CDCD66%22%2C+%2240%22%3A+%22%23006400%22%2C+%2250%22%3A+%22%2300A000%22%2C+%2260%22%3A+%22%23AAC800%22%2C+%2270%22%3A+%22%23003C00%22%2C+%2290%22%3A+%22%23286400%22%2C+%22100%22%3A+%22%23788200%22%2C+%22110%22%3A+%22%238CA000%22%2C+%22120%22%3A+%22%23BE9600%22%2C+%22130%22%3A+%22%23966400%22%2C+%22140%22%3A+%22%23FFB432%22%2C+%2